In [28]:
%%capture
%pip install opencv-python
%pip install tqdm
%pip install tensorflow
%pip install imageio
%pip install tensorflow_docs
%pip install scikit-learn
%pip install matplotlib

In [29]:
import random
import pathlib
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import itertools
from pathlib import Path
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import tensorflow as tf
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, RandomZoom


In [30]:
subset_paths = {}
subset_paths['train'] = Path('../Data/train')
subset_paths['test'] = Path('../Data/test')
subset_paths['val'] = Path('../Data/val')

In [31]:
def augment_frames(frames):
  # Flip horizontally with 50% chance
  frames = RandomFlip("horizontal")(frames)
  # Rotate randomly within +/- 10 degrees
  frames = RandomRotation(factor=0.2)(frames)
  # Adjust contrast between 0.8 and 1.2
  frames = RandomContrast(factor=0.2)(frames)
  # Zoom in/out randomly by 10%
  frames = RandomZoom(height_factor=0.1, width_factor=0.1)(frames)
  return frames

In [32]:
def format_frames(frame, output_size):
    """
      Pad and resize an image from a video.

      Args:
        frame: Image that needs to resized and padded.
        output_size: Pixel size of the output frame image.

      Return:
        Formatted frame with padding of specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

In [33]:

def frames_from_video_file(video_path, n_frames, output_size=(224, 224), frame_step=15):
    """
      Creates frames from each video file present for each category.

      Args:
        video_path: File path to the video.
        n_frames: Number of frames to be created per video file.
        output_size: Pixel size of the output frame image.

      Return:
        An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))
    src.release()
    result = np.array(result)[..., [2, 1, 0]]

    return result

In [34]:
class FrameGenerator:
    def __init__(self, path, n_frames, training=False):
        """ Returns a set of frames with their associated label.

          Args:
            path: Video file paths.
            n_frames: Number of frames.
            training: Boolean to determine if training dataset is being created.
        """
        self.path = path
        self.n_frames = n_frames
        self.training = training
        self.class_names = sorted(
            set(p.name for p in self.path.iterdir() if p.is_dir()))
        self.class_ids_for_name = dict((name, idx)
                                       for idx, name in enumerate(self.class_names))
        self.augment = augment_frames if training else None

    def get_files_and_class_names(self):
        video_paths = list(self.path.glob('*/*.mp4'))
        classes = [p.parent.name for p in video_paths]
        return video_paths, classes

    def __call__(self):
        video_paths, classes = self.get_files_and_class_names()
        pairs = list(zip(video_paths, classes))

        if self.training:
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = frames_from_video_file(path, self.n_frames)
            if self.augment:
                video_frames = self.augment(video_frames)
            label = self.class_ids_for_name[name]  # Encode labels
            yield video_frames, label

In [35]:
output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int16))

In [36]:
num_frames = 15

train_ds = tf.data.Dataset.from_generator(FrameGenerator(
    subset_paths['train'], num_frames, training=True),                                        output_signature=output_signature)

# toggle this augmentation
# train_ds = train_ds.map(lambda frames, label: (augment_frames(frames), label))

val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['val'], num_frames),
                                        output_signature=output_signature)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature=output_signature)

In [37]:
# the _frames is a 4D array to descibe the pixels in a "video" (not actually a video but rather a gorup of frames to represent a video) 
# val_frames[# of frame in the video burst][height of pixel][width of pixel][R:G:B value]
# ex. val_frames[2][0][0][2] will give the Blue value for the second frame in a video in the top left corner (since height = width = 0) and we will see the blue value (3rd in RGB)
print(f'shape of val_frames is ({num_frames}=number of frames 224=height in pixels 224=width in pixels 3=RBG value of the pixel )')

shape of val_frames is (15=number of frames 224=height in pixels 224=width in pixels 3=RBG value of the pixel )


fg = FrameGenerator(
    subset_paths['train'], 6, training=True)
output_signature

In [38]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


In [39]:
batch_size = 3
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

In [40]:
# net = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
net = tf.keras.applications.EfficientNetB0(include_top=False)
net.trainable = False

In [41]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=255),
    tf.keras.layers.TimeDistributed(net),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.GlobalAveragePooling3D(),
    tf.keras.layers.Dense(1)
])

In [42]:
model.compile(optimizer='adam',
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [43]:
model.fit(train_ds,
          epochs=2,
          validation_data=val_ds,
          callbacks=tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_accuracy'))

Epoch 1/2
96/96 [==============================] - 722s 6s/step - loss: 7.4502 - accuracy: 0.5087 - val_loss: 7.3936 - val_accuracy: 0.5152
Epoch 2/2
96/96 [==============================] - 431s 5s/step - loss: 7.4387 - accuracy: 0.5122 - val_loss: 7.3936 - val_accuracy: 0.5152


In [44]:
test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], 6),                                       output_signature=output_signature)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(2)
test_frames, test_labels = next(iter(test_ds))

In [45]:
y_pred  = model.predict(test_ds, verbose=1)
y_pred_bool = np.round(y_pred, 0)

112/112 [==============================] - 78s 617ms/step


In [46]:
test_loss, test_accuracy = model.evaluate(test_ds)

112/112 [==============================] - 69s 617ms/step - loss: 6.4458 - accuracy: 0.5580


In [47]:
test_accuracy

0.5580357313156128